In [36]:
import json
import collections
import numpy as np

from calculette_impots_m_language_parser.scripts import lighten_graph

The important variables are : ['NBPT', 'REVKIRE', 'BCSG', 'BRDS', 'IBM23', 'TXMOYIMP', 'NAPTIR', 'IINET', 'RRRBG', 'RNI', 'IDRS3', 'IAVIM']
Found 1435 unknown variables.
4740 formulas are used to compute an important variable.
233 constants are used to compute an important variable.
1463 inputs are used to compute an important variable.
782 undefined symbols are used to compute an important variable.


## Find cycles

### Adjacency matrix : too long (do not execute the following cells)

In [ ]:
index = {}
reverse_index = []
for i, k in enumerate(formulas_light):
    index[k] = i
    reverse_index.append(k)

In [ ]:
n = len(index)
adj = np.zeros((n, n))

In [ ]:
for parent in formulas_light:
    children = children_dict[parent]
    for child in children:
        if child in children_dict:
            adj[index[parent], index[child]] = 1

In [ ]:
current_power = 1
while current_power <= n:
    print(current_power)
    adj = np.dot(adj, adj)
    current_power += 1

In [ ]:
np.diag(adj).sum()

### Graph traversal : too long

In [ ]:
def cycle_research(node, genealogy):
    if node in genealogy:
        print('Cycle detected : %s'%str(genealogy))
    else:
        for child in children_light[node]:
            child_genealogy = list(genealogy)
            child_genealogy.append(node)
            cycle_research(child, child_genealogy)
        

In [ ]:
# do not execute this cell (too long)
cycle_research('IRN', [])

### Tarjan's strongly connected components algorithm

In [17]:
class State():
    def __init__(self):
        self.current_index = 0
        self.stack = []
        self.strongly_connected_components = []
        self.tarjan_indexes = {}
        self.low_link = {}
s = State()

In [18]:
def strong_connect(node, s):
    s.tarjan_indexes[node] = s.current_index
    s.low_link[node] = s.current_index
    s.current_index += 1
    s.stack.append(node)

    for child in children_light[node]:
        if child not in s.tarjan_indexes:
            strong_connect(child, s)
            s.low_link[node] = min(s.low_link[node], s.low_link[child])
        elif child in s.stack:
            s.low_link[node] = min(s.low_link[node], s.tarjan_indexes[child])

    if s.low_link[node] == s.tarjan_indexes[node]:
        new_component = []
        while s.stack[-1] != node:
            new_component.append(s.stack.pop())
        new_component.append(s.stack.pop())
        s.strongly_connected_components.append(new_component)

In [19]:
for node in formulas_light:
    if node not in s.tarjan_indexes:
        strong_connect(node, s)

In [20]:
component_size = [len(comp) for comp in s.strongly_connected_components]
c = collections.Counter(component_size)
c

Counter({1: 4740})

## Tree depth

In [37]:
# recursive : too long (do not execute)
def compute_depth(node):
    children_depth = [lighten_graph.compute_depth(child) for child in lighten_graph.children_light[node]]
    if children_depth:
        return 1 + max(children_depth)
    else:
        return 1

In [38]:
depths = {}
for var in lighten_graph.computing_order:
    children_depth = [depths[child] for child in lighten_graph.children_light[var]]
    if children_depth:
        depths[var] = 1 + max(children_depth)
    else:
        depths[var] = 1

In [39]:
depths['IINET']

1119